In [1]:
'''
测试数据准备
'''

import os
import pickle
from tqdm import tqdm
from collections import defaultdict
import argparse
from core.models.entailment import EntailmentDeberta
from core.data.data_utils import load_ds_from_json

def load_pickle_file(file_path):
    with open(file_path, 'rb') as f:
        data = pickle.load(f)
    return data

def save_pickle_file(file_path, data):
    with open(file_path, 'wb') as f:
        pickle.dump(data, f)


In [2]:
result = load_pickle_file("output/result/Qwen/Qwen2.5-7B-Instruct/squad_train.pkl")
print(len(result["id"]))
print(result["data"][result["id"][0]].keys())

10000
dict_keys(['example', 'golden', 'irrelevant', 'without'])


In [3]:
d = result["data"][result["id"][0]]
d.keys()

dict_keys(['example', 'golden', 'irrelevant', 'without'])

In [4]:
d['example']

{'id': '57313c0a497a881900248cae',
 'question': 'How were pigments like red lake created?',
 'context': 'Unlike vermilion or red ochre, made from minerals, red lake pigments are made by mixing organic dyes, made from insects or plants, with white chalk or alum. Red lac was made from the gum lac, the dark red resinous substance secreted by various scale insects, particularly the Laccifer lacca from India. Carmine lake was made from the cochineal insect from Central and South America, Kermes lake came from a different scale insect, kermes vermilio, which thrived on oak trees around the Mediterranean. Other red lakes were made from the rose madder plant and from the brazilwood tree.',
 'answers': ['mixing organic dyes'],
 'irrelevant_id': '57268318708984140094c88b',
 'irrelevant_context': 'One of the paper\'s best known front pages, published on 4 May 1982, commemorated the torpedoing of the Argentine ship the General Belgrano by running the story under the headline "GOTCHA". At MacKenzie

In [8]:
d['golden'].keys()

dict_keys(['greedy', 'sample'])

In [10]:
d['golden']['greedy'].keys()

dict_keys(['text', 'slt_embedding'])

In [11]:
d['golden']['greedy']['text']

'Pigments like red lake were created by mixing organic dyes from insects or plants with white chalk or alum.'

In [12]:
d['golden']['greedy']['slt_embedding'].shape

torch.Size([29, 1, 3584])

In [15]:
len(d['golden']['sample'])

10

In [16]:
d['golden']['sample'][0]

{'text': 'Pigments like red lake were created by mixing organic dyes from insects or plants with white chalk or alum.',
 'slt_embedding': None,
 'cluster_id': 0}